<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [70]:
import pandas as pd
import numpy as np
import csv

file_nested_list = []

# reading the csv into an object
with open('data/chipotle.tsv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for r in reader:
        file_nested_list.append(r)

# reading the chipotle csv into a pandas dataframe
df = pd.read_csv('data/chipotle.tsv', '\t')
df.head(15)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


How many orders are being made?

In [71]:
# counting total orders by getting the number of unique order_ids
# this should account for any possible skipped order number
num_orders = len(df.order_id.unique())
print(num_orders)

1834


### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [72]:
#header is the first item in the list
header = file_nested_list[0]

#data is the rest of the lists
data = file_nested_list[1:]

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [83]:
#get list of unique orders
order_nums = df.order_id.unique()

#get total price for each unique order
order_prices = []
for order in order_nums:
    items = df[df.order_id == order]
    #multiply each price value by quantity
    prices = pd.to_numeric(items.item_price.replace('[\$,)]','', regex=True)) * items.quantity
    total = prices.sum()
    order_prices.append(total)

# get average of prices
average_price = np.mean(order_prices)
average_price

21.394231188658669

### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [106]:
#limit to rows with sodas/soft drinks
sodas = df[(df.item_name == 'Canned Soda') | (df.item_name == 'Canned Soft Drink')]

#get unique values, and convert to list
unique_sodas = sodas.choice_description.unique().tolist()
unique_sodas

['[Sprite]',
 '[Dr. Pepper]',
 '[Mountain Dew]',
 '[Diet Dr. Pepper]',
 '[Coca Cola]',
 '[Diet Coke]',
 '[Coke]',
 '[Lemonade]',
 '[Nestea]']

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [135]:
#assuming Burrito always spelled with capital 'B', limit to rows with Burritos
burritos = df[df.item_name.str.contains('Burrito')]

#split each value into a list of toppings
toppings = burritos.choice_description.str.split(',')

# get the length of each list, then the mean of those lengths
average_toppings = toppings.str.len().mean()
average_toppings

5.3950511945392492

### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [143]:
#assuming all chip orders contain 'Chips' spelled with a capital 'C'
#limit to chip order rows
chip_series = df[df.item_name.str.contains('Chips')]

#get list of unique chip order types
unique_chips = chip_series.item_name.unique().tolist()

# get sum of quantity columns for each type of chip order
chip_orders = {}
for order in unique_chips:
    chip_orders[order] = chip_series[chip_series.item_name == order].quantity.sum()
    
chip_orders
    


{'Chips': 230,
 'Chips and Fresh Tomato Salsa': 130,
 'Chips and Guacamole': 506,
 'Chips and Mild Fresh Tomato Salsa': 1,
 'Chips and Roasted Chili Corn Salsa': 23,
 'Chips and Roasted Chili-Corn Salsa': 18,
 'Chips and Tomatillo Green Chili Salsa': 45,
 'Chips and Tomatillo Red Chili Salsa': 50,
 'Chips and Tomatillo-Green Chili Salsa': 33,
 'Chips and Tomatillo-Red Chili Salsa': 25,
 'Side of Chips': 110}

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
